# Modifying Models

Given a model that is already built and possibly optimized, the user might want to modify single constraint or variable bounds by means of correction or exploration of the feasible space. 

In the following we show how single elements can be tweaked or rewritten. Let's start with the simple model of the `Getting Started` section. 

In [ ]:
import pandas as pd
import xarray as xr

import linopy

In [ ]:
m = linopy.Model()
time = pd.Index(range(10), name="time")

x = m.add_variables(
    lower=0,
    coords=[time],
    name="x",
)
y = m.add_variables(lower=0, coords=[time], name="y")

factor = pd.Series(time, index=time)

con1 = m.add_constraints(3 * x + 7 * y >= 10 * factor, name="con1")
con2 = m.add_constraints(5 * x + 2 * y >= 3 * factor, name="con2")

m.add_objective(x + 2 * y)
m.solve()

m.solve()
sol = m.solution.to_dataframe()
sol.plot(grid=True, ylabel="Optimal Value")

The figure above shows the optimal values of `x(t)` and `y(t)`. 

## Varying lower and upper bounds

Now, let's say we want to set the lower bound of `y(t)` to 1. This would translate to:

In [ ]:
x.lower = 1

.. note::
   The same could have been achieved by calling `m.variables.x.lower = 1`

Let's solve it again!

In [ ]:
m.solve()
sol = m.solution.to_dataframe()
sol.plot(grid=True, ylabel="Optimal Value")

In [ ]:
sol

We see that the new lower bound of x is binding across all time steps.

Of course the implementation is flexible over the dimensions, so we can pass non-scalar values:

In [ ]:
x.lower = xr.DataArray(range(10, 0, -1), coords=(time,))

In [ ]:
m.solve()
sol = m.solution.to_dataframe()
sol.plot(grid=True, ylabel="Optimal Value")

You can manipulate the upper bound of a variable in the same way.

## Varying Constraints

A similar functionality is implemented for constraints. Here we can modify the left-hand-side, the sign and the right-hand-side.

Assume we want to relax the right-hand-side of the first constraint `con1` to `8 * factor`. This would translate to:

In [ ]:
con1.rhs = 8 * factor

.. note::
   The same could have been achieved by calling `m.variables.con1.rhs = 8 * factor`

Let's solve it again!

In [ ]:
m.solve()
sol = m.solution.to_dataframe()
sol.plot(grid=True, ylabel="Optimal Value")

In contrast to previous figure, we now see that the optimal value of `y` does not reach values above 10 in the end. 

In the same way, we can modify the left-hand-side. Assume we want to weight `y` with a coefficient of 8 in the constraints, this gives

In [ ]:
con1.lhs = 3 * x + 8 * y

**Note:**
The same could have been achieved by calling 
```python 
m.constraints['con1'].lhs = 3 * x + 8 * y
```

which leads to

In [ ]:
m.solve()
sol = m.solution.to_dataframe()
sol.plot(grid=True, ylabel="Optimal Value")

## Varying the objective 

Varying the objective happens in the same way as for the left-hand-side of the constraint as it is a linear expression too. Note, when passing an unstacked linear expression, i.e. an expression with more than the `_term` dimension, `linopy` will automatically stack it. 

So assume, we would like to modify the weight of `y` in the objective function, this translates to:

In [ ]:
m.objective = x + 3 * y

In [ ]:
m.solve()
sol = m.solution.to_dataframe()
sol.plot(grid=True, ylabel="Optimal Value")

As a consequence, `y` stays at zero for all time steps.

In [ ]:
m.objective